Files taken from github: **airports, routes, airline**

In [1]:
import pandas as pd

In [2]:
def get_github_data(link: str, file_type: str = "csv") -> pd.DataFrame:
    
    '''
    Load in data from a raw github link. Accepts both .csv and .dat files.
    
    Parameters
    ----------
    link : string, required
        DESCRIPTION. A link for the raw github content.

    Returns
    -------
    A DataFrame with the link's content.

    '''
    if file_type == 'csv':
        return pd.read_csv(link)
    elif file_type == 'dat':
        return pd.read_csv(link, header=None)
    else:
        raise Exception('Unsupported file format. Try either: csv, dat')

In [3]:
# folder to save into
save_path = 'cleaned_data/'

# Airports

In [4]:
#reference: GitHub David Meg 
#https://github.com/davidmegginson/ourairports-data)

df_airports = get_github_data('https://raw.githubusercontent.com/davidmegginson/ourairports-data/main/airports.csv')

In [5]:
df_airports['type'].value_counts()

small_airport     38641
heliport          17986
closed             8972
medium_airport     4747
seaplane_base      1107
large_airport       451
balloonport          35
Name: type, dtype: int64

In [6]:
# filter to large
df_airports = df_airports[df_airports['type']=='large_airport']
df_airports['iso_country'].value_counts()

US    70
CN    36
RU    17
IT    14
JP    12
      ..
RO     1
ME     1
MZ     1
SV     1
SN     1
Name: iso_country, Length: 159, dtype: int64

In [7]:
df_airports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 451 entries, 11087 to 71929
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 451 non-null    int64  
 1   ident              451 non-null    object 
 2   type               451 non-null    object 
 3   name               451 non-null    object 
 4   latitude_deg       451 non-null    float64
 5   longitude_deg      451 non-null    float64
 6   elevation_ft       446 non-null    float64
 7   continent          340 non-null    object 
 8   iso_country        450 non-null    object 
 9   iso_region         451 non-null    object 
 10  municipality       444 non-null    object 
 11  scheduled_service  451 non-null    object 
 12  gps_code           445 non-null    object 
 13  iata_code          444 non-null    object 
 14  local_code         103 non-null    object 
 15  home_link          273 non-null    object 
 16  wikipedia_link     4

In [8]:
df_airports.drop(["ident", "type", "latitude_deg", "longitude_deg", "elevation_ft", "continent", "iso_region", "municipality", "gps_code", "iata_code", "local_code", "home_link", "wikipedia_link", "keywords", "id"], axis=1, inplace=True)
df_airports.reset_index(inplace=True, drop=True)
df_airports.rename(columns={'iso_country': 'country_id', 'scheduled_service': 'active'}, inplace=True)
df_airports.head()



,name,country_id,active
0,Honiara International Airport,SB,yes
1,Momote Airport,PG,yes
2,Port Moresby Jacksons International Airport,PG,yes
3,Keflavik International Airport,IS,yes
4,Edmonton International Airport,CA,yes


**Export to CSV**

In [9]:
df_airports.to_csv(save_path + 'airports.csv')
df_airports.to_parquet(save_path + 'airports.parquet')

# Routes

In [10]:
#reference Jani Patokallio routes, airlines 
#https://github.com/jpatokal/openflights/

df_routes = get_github_data('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat', 'dat')

In [11]:
df_routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       67663 non-null  object
 1   1       67663 non-null  object
 2   2       67663 non-null  object
 3   3       67663 non-null  object
 4   4       67663 non-null  object
 5   5       67663 non-null  object
 6   6       14597 non-null  object
 7   7       67663 non-null  int64 
 8   8       67645 non-null  object
dtypes: int64(1), object(8)
memory usage: 4.6+ MB


In [12]:
columns = 'airline airline_id source_airport source_airport_id destination_airport destination_airport_id codeshare stops equipment'.split()
df_routes.columns = columns

In [13]:
df_routes.drop(["airline", "source_airport", "destination_airport","codeshare", "stops", "equipment"], axis=1, inplace=True)
df_routes.rename(columns={'source_airport_id':'orgin_airport_id'}, inplace=True)
df_routes.head()

,airline_id,orgin_airport_id,destination_airport_id
0,410,2965,2990
1,410,2966,2990
2,410,2966,2962
3,410,2968,2990
4,410,2968,4078


**Export to CSV**

In [14]:
df_routes.to_csv(save_path + 'routes.csv')
df_routes.to_parquet(save_path + 'routes.parquet')

# Airlines

In [15]:
#reference Jani Patokallio routes, airlines 
#https://github.com/jpatokal/openflights/

df_airlines = get_github_data('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat', 'dat')

In [16]:
columns = 'airline_id name alias IATA ICAO callsign country active'.split()
df_airlines.columns = columns

In [17]:
df_airlines.drop(["alias", "IATA","ICAO", "callsign", "active"], axis=1, inplace=True)
#add country_id column
df_airlines

,airline_id,name,country
0,-1,Unknown,\N
1,1,Private flight,NaN
2,2,135 Airways,United States
3,3,1Time Airline,South Africa
4,4,2 Sqn No 1 Elementary Flying Training School,United Kingdom
...,...,...,...
6157,21248,GX Airlines,China
6158,21251,Lynx Aviation (L3/SSX),United States
6159,21268,Jetgo Australia,Australia
6160,21270,Air Carnival,India


**Export to CSV**

In [18]:
df_airlines.to_csv(save_path + 'airlines.csv')
df_airlines.to_parquet(save_path + 'airlines.parquet')

# Countries

In [19]:
#reference Luke Duncalfe countries
#https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes

df_countries = get_github_data('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv')

In [20]:
df_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      249 non-null    object 
 1   alpha-2                   248 non-null    object 
 2   alpha-3                   249 non-null    object 
 3   country-code              249 non-null    int64  
 4   iso_3166-2                249 non-null    object 
 5   region                    248 non-null    object 
 6   sub-region                248 non-null    object 
 7   intermediate-region       107 non-null    object 
 8   region-code               248 non-null    float64
 9   sub-region-code           248 non-null    float64
 10  intermediate-region-code  107 non-null    float64
dtypes: float64(3), int64(1), object(7)
memory usage: 21.5+ KB


In [21]:
df_countries.drop(["country-code", "iso_3166-2", "sub-region", "intermediate-region", "region-code", "sub-region-code", "intermediate-region-code"], axis=1, inplace=True)
df_countries.rename(columns={'alpha-2':'ISO_code_2','alpha-3':'ISO_code_3'}, inplace=True)



df_countries.head()

,name,ISO_code_2,ISO_code_3,region
0,Afghanistan,AF,AFG,Asia
1,Åland Islands,AX,ALA,Europe
2,Albania,AL,ALB,Europe
3,Algeria,DZ,DZA,Africa
4,American Samoa,AS,ASM,Oceania


**Export to CSV**

In [22]:
df_countries.to_csv(save_path + 'countries.csv')
df_countries.to_parquet(save_path + 'airlines.parquet')